# Web Components

En el capítulo del DOM hemos introducido gran parte de los conocimientos necesarios para este capítulo. No obstante, necesitamos haber trabajado con todos los demás conceptos y haber practicado con proyectos simples para entender la importancia de los `Web Components`. 

Si queremos hacer una SPA que maneje datos obtenidos del servidor y los muestre de manera reactiva, podemos seguir estos pasos:

* El router manda descargar los datos del servidor y los pasa a una función que los aplica a una plantilla. 
* El router se suscribe a un Observable de los datos del servidor y cada vez que llegan nuevos datos los renderiza con una plantilla.
* El router renderiza una plantilla con "placeholders", espera los datos del servidor y rellena la plantilla con esos datos. 
* ...

En cualquiera de estas opciones, hay problemas para volver a renderizar cuando hay datos nuevos, para mantener a raya a las suscripciones, para gestionar el estado de la aplicación o para mantenerla suficientemente desacoplada. Además, será complicado reutilizar esas funciones en otros proyectos.  

Los `Web Components` no son una tecnología diferente a las vistas en los capítulos anteriores, se trata de una definición de la W3C para estandarizar lo que comenzaba a ser un aspecto común de los frameworks e intentar hacer componentes reutilizables para todos. Estos componentes se pueden hacer a partir de las mejoras de ES6 y HTML5. En la definición que podemos encontrar en la web de MDN: https://developer.mozilla.org/en-US/docs/Web/API/Web_components, no incorpora la reactividad, pero nosotros vamos a incorporarla para hacerlos más útiles en el curso. 

> Llegado el momento de hacer Web Components desde 0 con "vanilla Javascript" nos podemos preguntar si no será mejor usar un framework. Intentemos resistir esa tentación, al menos para aprender, porque esos conocimientos nos permitirán entender profundamente cómo funciona, por ejemplo, Angular o React. 


## Conceptos iniciales

Se pueden crear componentes de muchas maneras, pero la definición oficial parte de la posibilidad de crear **Elementos personalizados** o `Custom Elements`. Esto se puede hacer porque Javascript permite heredar de `HTMLElement`: https://developer.mozilla.org/en-US/docs/Web/API/Web_components/Using_custom_elements. Luego veremos las posibilidades que hay con esto. 

Los elementos personalizados pueden ser diferentes al resto de la aplicación y tener sus propios scripts y estilos. Estos pueden colisionar con una aplicación para la que no fueron creados desde el principio. Una de las necesidades cubiertas por los Web Components es la reutilización del código. Pero Javascript permite encapsular estos elementos en un DOM separado del principal. Esta técnica se llama `Shadow DOM`. 

Para crear elementos personalizados en el `Shadow DOM`, se pueden hacer programáticamente o a través del HTML con etiquetas como `<template>` o `<slot>`. 

Por consiguiente, un `Web Component` tiene una clase que hereda de `HTMLElement` registrada en el `CustomElementRegistry` para que pueda ser utilizada en cualquier parte. Este elemento tiene un `Shadow DOM` (opcional) para que su código y estilos no molesten al resto y se suele usar `<template>` para las plantillas. A continuación se puede usar como cualquier etiqueta. 

### Alternativas

Como todo en Javascript, es opcional usar Web components, pero es que también es opcional usar todas las técnicas anteriores. Si no hacemos un `Custom Element` no entra en la definición de lo que estamos tratando, pero se puede hacer una función que retorne un elemento con `Shadow DOM` y código personalizado que atiende a los eventos, descarga los datos, se suscribe a Observables... El problema es que perdemos las ventajas del ciclo de vida de los `HTMLElements`. 

Tampoco es necesario siempre hacer `Shadow DOM` si el código o estilo no va a interferir nunca. 

Por otro lado, el uso de `<template>` puede ser muy farragoso en algunos casos comparado con las `Template Literals`. En nuestro caso, haremos un uso combinado de los mismos para aprovechar las ventajas de ambas técnicas. 

## Custom Elements

Se trata de elementos HTML que tienen un comportamiento definido por el desarrollador. Una vez registrados, quedan disponibles en el navegador en esa página web.

Ya existen algunos creados como `HTMLImageElement` o `HTMLParagrafElement` que tienen un comportamiento extendido respecto al estándar de los elementos comunes. Pero a nosotros nos interesa crear nuevos desde cero. 

Para crearlo tan solo hay que extender la clase:

```javascript
class PopupInfo extends HTMLElement {
  constructor() {
    super();
  }
  // Element functionality written in here
}
```

En el constructor se inicializa el elemento con valores por defecto, registro de eventos, creación del `Shadow Root` y poco más. Dejaremos la creación de elementos hijos o añadir atributos a estos para después, en otras etapas del ciclo de vida. 

### Ciclo de vida

Los elementos personalizados cuentan con una serie de funciones que pueden ser implementadas y que son invocadas a lo largo de su ciclo de vida. Estas son:

* connectedCallback(): Se llama cada vez que se agrega el elemento al documento. Se recomienda configurar el elemento en este punto, mejor que en el constructor. 
* disconnectedCallback(): Se llama cada vez que se elimina el elemento del documento.
* adoptedCallback(): Se llama cada vez que se mueve el elemento a un nuevo documento.
* attributeChangedCallback(): se llama cuando se cambian, agregan, eliminan o reemplazan atributos. 

Podemos probar este código en la consola del navegador: 

```javascript
class MyCustomElement extends HTMLElement {
  static observedAttributes = ["class", "size"];

  constructor() {
    super();
      console.log("Constructor")
  }

  connectedCallback() {
    console.log("Custom element added to page.");
  }

  disconnectedCallback() {
    console.log("Custom element removed from page.");
  }

  adoptedCallback() {
    console.log("Custom element moved to new page.");
  }

  attributeChangedCallback(name, oldValue, newValue) {
    console.log(`Attribute ${name} has changed from ${oldValue} to ${newValue}.`);
  }
}

customElements.define("my-custom-element", MyCustomElement);
// La prueba
let customElement = document.createElement('my-custom-element');
console.log('Creado');
document.body.append(customElement);
console.log('Añadido');
customElement.classList.add('customClass','customClass2');
console.log('Cambio del atributo class');
customElement.remove();
console.log('Eliminado');
```

Además de esas funciones del ciclo de vida, se pueden añadir otras. Una típica puede ser la función `render()` que retorna el elemento redibujado. Esto es útil si queremos que el contenido pueda ser actualizado reactivamente. 

### Registrar el elemento personalizado

Como se puede ver en el ejemplo anterior, lo hemos registrado con:

```javascript
customElements.define("my-custom-element", MyCustomElement);
```

Es importante recalcar que el nombre es preciso que tenga **un guión (-)** en medio del nombre para que Javascript no los confunda. 

### Usar el elemento personalizado

Se puede usar de forma normal como cualquier etiqueta HTML:

```html
<my-custom-element></my-custom-element>
```

## Shadow DOM

Como ya hemos visto en la introducción, el objetivo del `Shadow DOM` es encapsular el comportamiento del elemento personalizado. 

A partir de la raíz de DOM principal de la página web cuelgan todos los nodos de forma jerárquica. El Shadow DOM permite registrar árboles ocultos a partir de un nodo del árbol principal. Estos árboles empiezan por el `Shadow Root`. Esta raíz está unida al árbol principal mediante un nodo que actúa de `Shadow Host`. 

Se pueden crear de forma imperativa mediante Javascript:

```javascript
const host = document.querySelector("#host");
const shadow = host.attachShadow({ mode: "open" });
const span = document.createElement("span");
span.textContent = "I'm in the shadow DOM";
shadow.appendChild(span);
```

También se puede crear de forma declarativa mediante HTML:

```html
<div id="host">
  <template shadowrootmode="open">
    <span>I'm in the shadow DOM</span>
  </template>
</div>
```

La segunda manera permite delegar la creación del `Shadow DOM` al servidor, al enviar este el HTML ya sea dinámica o estáticamente. 

Observemos que en los dos casos tenemos la palabra `open`, esta permite que el `Shadow DOM` sea accesible desde la web. Si está en `close`, se verá, pero no será accesible mediante selectores. 

Si no ponemos el `shadowrootmode` en el `template` no se verá, si lo ponemos, ya sea open o close, se verá pero será o no accesible. Esto no es compatible con navegadores antiguos.   

Llegados a este punto, podemos poner esto en práctica copiando y pegando este código en la consola de cualquier web que tengamos abierta:


```javascript
const body = document.querySelector("body");
const host = document.createElement('div');
body.append(host);
const shadow = host.attachShadow({ mode: "closed" });
const span = document.createElement("span");
span.textContent = "I'm in the shadow DOM from Javascript";
shadow.append(span);

const host2 = document.createElement('div');
host2.innerHTML = `
  <template shadowrootmode="open">
    <span>I'm in the shadow DOM from HTML</span>
  </template>
`;
body.append(host2);
```

El segundo no se va a ver porque innerHTML no crea el `Shadow Root`. Este, por javascript, ha de ser creado explícitamente como en el primer ejemplo. No obstante, aunque no se vea, el template queda accesible y puede ser clonado para incorporarlo más adelante. 

No es posible que un `Element` que actua como `shadow host` tenga tanto shadow-root como elementos hijos en el árbol principal. No da error, pero no serán visibles. De la misma manera, cualquier `div` o otros elementos que contengan `<template>` se convierte automáticamente en un `shadow host`, por lo que no puede tener otras etiquetas normales dentro. 

los nodos descencientes del shadow-root no son accesibles mediante `document.querySelector`, por ejemplo. No obstante, son accesibles a través del atributo `shadowRoot`, el cual permite ejecutar `querySelector` internamente.

```javascript
host.shadowRoot.querySelector("span");
```
#### Estilos en shadow DOM

Para dar estilo a un shadow DOM se puede hacer modificando `.style...` en los elementos hijos, poniendo la etiqueta `<style>` y escribiendo el CSS ahí mismo o importando los ficheros CSS:



Los estilos generales no se aplican a los elementos dentro del `shadow root`. Pero si queremos que sí lo hagan todos, podemos usar un código como este:

```javascript
const adoptedStyleSheets = new CSSStyleSheet();        
const rules = [...document.styleSheets].flatMap(s => [...s.cssRules].map(r => r.cssText)).join(' ');
adoptedStyleSheets.replace(rules);
shadow.adoptedStyleSheets = [adoptedStyleSheets];
```
Aquí se ha obtenido la lista de reglas de todas las hojas de estilo y se han añadido al `shadow`. Para ello hay que transformar mediante flatMap y demás la lista de estilos, dentro de ellos las reglas y dentro las reglas en string, crear un `CSSStyleSheet` personalizado y añadir todas las reglas. 

También se podrian buscar solo las reglas de un determinado tipo, una regla en concreto o las que vienen en un fichero. 

## Templates y Slots

Ya hemos visto en el apartado anterior que se pueden crear `Shadow Root` desde una `<template>`. En los ejemplos anteriores eran usados con ese propósito, pero la etiqueta `<template>` también puede ser usada para hacer plantillas que no se van a ver hasta que se clonen.

Suponiendo que definimos esta plantilla:

```html
<template id="custom-paragraph">
  <p>My paragraph</p>
</template>
```

Podemos crear un `Web Component` de esta manera:

```javascript
customElements.define(
  "my-paragraph",
  class extends HTMLElement {
    constructor() {
      super();
      let template = document.getElementById("custom-paragraph");
      let templateContent = template.content;

      const shadowRoot = this.attachShadow({ mode: "open" });
      shadowRoot.appendChild(templateContent.cloneNode(true));
    }
  }
);
```

Observemos que estamos extendiendo la clase `HTMLElement` dentro de la misma función `customElements.define()`, que creamos explícitamente el `Shadow Root` y que le añadimos un clon de la `<template>`.

Es posible crear plantillas con `<template>` que puedan ser modificadas después de ser clonadas. Se puede hacer manualmente buscando dentro de ellas los elementos y cambiando su `innerText` o `value`. Por ejemplo, si tenemos que crear una tabla y poner datos en cada fila, podemos tener una plantilla para las filas y, en este caso, para la última columna:

```html
<template id="tableBodyTR">
  <tr>
    <th scope="row">{id}</th>
    <td>{ejemplo}</td>
  </tr>
</template>
<template id="lastColumn">
  <td class="buttonsCol">
    <span class="edit">📝</span><span class="delete">🗑️</span>
  </td>
</template>
```

```javascript
let lastColumnTemplate = div.querySelector("#lastColumn");
let tableBody = div.querySelector("#tableBody");
let tableBodyTR = div.querySelector("#tableBodyTR");

for (let row of datos) {
  let tableBodyTRRow = tableBodyTR.content.cloneNode(true).querySelector("tr");
  tableBodyTRRow.innerHTML = columns
    .map((col) => `<td data-col="${col}">${row[col]}</td>`)
    .join("");
  let lastColumn = lastColumnTemplate.content
    .cloneNode(true)
    .querySelector("td");
  lastColumn.dataset.id = row.id;
  tableBodyTRRow.append(lastColumn);
  tableBody.append(tableBodyTRRow);
}
```

En el ejemplo anterior, la primera columna de la plantilla `tableBodyTR` desaparece por el innerHTML. 

> Para usar `<template>` no es necesario estar en un `Web Component`. Se puede usar en cualquier momento, por eso también está explicado en el capítulo del DOM. 

### Slots

En el ejemplo anterior esas sustituciones se hacen manualmente, algunas veces se puede hacer más elegantemente con `<slot>`. La etiqueta debe tener un `name` y permite se sustituida. 

Dentro de la plantilla se puede poner esta etiqueta de esta manera:

```html
<p><slot name="my-text">My default text</slot></p>
```

Y será sustituida en HTML así:

```html
<my-paragraph>
  <span slot="my-text">Let's have some different text!</span>
</my-paragraph>
```

### Alternativa con Template Literals

Usar `<template>` implica tener que buscarlo, clonarlo y modificarlo. En ocasiones es mucho más simple el uso de `Template Literals`. Incluso se puede hacer con `Tagged Template Literals` y queda un código más compacto. En el capítulo del DOM hay un ejemplo: https://xxjcaxx.github.io/libro_dwec/dom.html#creacion-de-elementos-mediante-tagged-template-literals 




## Ejemplo completo

En el siguiente ejemplo se muestra la creación de un componente que muestra la tabla `profiles`de `Supabase`. Además de mostrar la tabla, tiene la posibilidad de ocultar columnas y mostrar un formulario para editar cada fila. No está implementada toda la funcionalidad de editar o eliminar, pero el resto funciona. Además, usamos conceptos un poco avanzados como el conseguir la reactividad mediante un `BehaviorSubject` o usar una función de composición. 

En el `constructor` se establecen unos datos estáticos de ejemplo mientras cargan los datos del servidor. 

En la función `connectedCallback` se suscribe al `BehaviorSubject`, descarga los datos para que, cuando estén, actualice el `BehaviorSubject`. 

Es importante no dejar cabos sueltos cuando se elimina el componente, por eso se anula la subscripción en `disconnectedCallback`.

Se ha creado la función `render(state)` que renderiza el estado. Esta es interesante porque, usando una función de composición, vamos aplicando una plantilla hecha con algunos `<template>`, a continuación se introducen los datos en la plantillas, en la siguiente función de la composición se añaden los eventos y en la siguiente algunos posibles efectos. Con esto hemos conseguido separar, aunque no totalmente, la plantilla de los datos y de los eventos. 

En este caso se ha optado por no usar `Shadow DOM` porque interesaba poder usar sin problemas los estilos y javascript de boostrap que tenemos en la aplicación principal. 

```javascript
export class CustomTable extends HTMLElement {

  constructor() {
    super();
    this.stateSubject = new BehaviorSubject({
      data: [
        { id: 0, name: `placeholder` },
        { id: 1, name: `placeholder2` }
      ],
      columns: new Set(['id','name']),
      hiddenColumns: new Set([]),
      showHiddenColumns: false,
      filters: [],

    });
    this.stateSubscription = null;
  }

  connectedCallback() {
    this.append(this.render(this.stateSubject.value));
    this.stateSubscription = this.stateSubject.subscribe(state => {
      this.firstElementChild.remove();
      this.append(this.render(state));
    });
    let dataPromise = getDataSupabase("profiles", "");
    dataPromise.then(data => {
      let columns = Object.keys(data[0]);
      this.stateSubject.next({
        data: data,
        columns: new Set(columns),
        hiddenColumns: new Set([]),
        filters: []
      })
    });
  }

  disconnectedCallback() {
    this.stateSubscription?.unsubscribe();
  }

  render(state) {
    return _.compose(
      (div) => { // Añadir efectos

        if(state.showHiddenColumns){
          div.querySelector('#collapseColumnas').classList.add("show");
        }
        else {
          div.querySelector('#collapseColumnas').classList.remove("show");
        }

      return div;
      },
      
      (div) => { // Añadir eventos
        div.addEventListener("click", (event) => {
          if (event.target.className === "edit") {
            let id = event.target.parentNode.dataset.id;
            let tds = [...event.target.parentNode.parentNode.querySelectorAll("td")].slice(0, -1);
            tds.forEach(td => {
              let content = td.innerText;
              td.innerHTML = `<input type="text" name="${td.dataset.col}" value="${content}"/>`
            });
            let saveButton = document.createElement('span');
            saveButton.innerHTML = '💾';
            event.target.parentNode.prepend(saveButton);
            saveButton.addEventListener("click", () => {
              console.log(tds.map(td => td.dataset.col));
            });
          }

          if (event.target.className === "delete") {
            let id = event.target.parentNode.dataset.id;
            let confirm = window.confirm("Vas a borrar ¿Estas seguro?");
            if (confirm) {
              console.log(event.target.parentNode.dataset.id);
            }
          }

          if(event.target.dataset.column){
            const newState = structuredClone(state);
            newState.hiddenColumns.add(event.target.dataset.column);
            newState.showHiddenColumns = true;
            this.stateSubject.next(newState);
          }
        });
        return div;
      },
      (div) => {  // Mostrar datos
        let divColumnas = div.querySelector("#divColumnas");
        let checkColumnasTemplate = div.querySelector("#checkColumnasTemplate");
        let tableHeader = div.querySelector("#tableHeader");
        let tableHeaderTH = div.querySelector("#tableHeaderTH");
        let lastColumnTemplate = div.querySelector("#lastColumn");
        let tableBody = div.querySelector("#tableBody");
        let tableBodyTR = div.querySelector("#tableBodyTR");

      
        let data = state.data;
        let columns = [...state.columns.difference(state.hiddenColumns)];
 

         for (let column of state.columns) {
            let checkColumn = checkColumnasTemplate.content
              .cloneNode(true)
              .querySelector("div");
            let label = checkColumn.querySelector("label");
            label.innerText = column;
            let checkbox = checkColumn.querySelector("input");
            checkbox.checked = state.hiddenColumns.has(column);
            checkbox.id = `hideColumn${column}`;
            checkbox.dataset.column = column;
            label.setAttribute('for',checkbox.id);
            divColumnas.append(checkColumn);
          }
            for (let column of columns) {
            let tableHeaderTHColumn = tableHeaderTH.content
              .cloneNode(true)
              .querySelector("th");
            tableHeaderTHColumn.innerText = column;
            tableHeader.append(tableHeaderTHColumn);
          }
          let lastColumn = lastColumnTemplate.content.cloneNode(true).querySelector("td");
          tableHeader.append(lastColumn);

          for (let row of data) {
            let tableBodyTRRow = tableBodyTR.content
              .cloneNode(true)
              .querySelector("tr");
            tableBodyTRRow.innerHTML = columns.map(col => `<td data-col="${col}">${row[col]}</td>`).join('');
            let lastColumn = lastColumnTemplate.content.cloneNode(true).querySelector("td");
            lastColumn.dataset.id = row.id;
            tableBodyTRRow.append(lastColumn);
            tableBody.append(tableBodyTRRow);

          }
       // });
        return div;
      },
      DOM.createDivWithInnerHTML // Genera el div
    )(/* HTML */ `
      <div class="container bd-gutter mt-3 my-md-4 bd-layout overflow-scroll">
     
        <h2 class="d-inline-flex gap-1">Alumnos</h2>
  
        <p class="d-inline-flex gap-1">
          <button
            class="btn btn-primary"
            type="button"
            data-bs-toggle="collapse"
            data-bs-target="#collapseColumnas"
            aria-expanded="false"
            aria-controls="collapseColumnas"
          >
            Columnas
          </button>
        </p>
        <div class="collapse" id="collapseColumnas">
          <div class="card card-body" id="divColumnas">
            <template id="checkColumnasTemplate">
              <div class="form-check form-switch">
                <input
                  class="form-check-input"
                  type="checkbox"
                  role="switch"
                  id="columnasSwitchtemplate"
                />
                <label class="form-check-label" for="columnasSwitchtemplate"
                  >Default switch checkbox input</label
                >
              </div>
            </template>
          </div>
        </div>   
        <table class="table table-striped table-hover">
          <thead>
            <tr id="tableHeader">
              <template id="tableHeaderTH">
                <th scope="col">#</th>
              </template>
            </tr>
          </thead>
          <tbody id="tableBody">
            <template id="tableBodyTR">
              <tr>
                <th scope="row">{id}</th>
                <td>{exemple}</td>
              </tr>
            </template>
            <template id="lastColumn">
            <td class="buttonsCol"><span class="edit">📝</span><span class="delete">🗑️</span></td>
            </template>
          </tbody>
        </table>
      </div>
    `);
  }
}
```

A continuación, el fichero principal de la aplicación importará este `Web Component` y lo registrará para luego poder usarlo:

```javascript
import { CustomTable } from './vistaAdministrador';
...
window.customElements.define('custom-table', CustomTable);
...
container.append(document.createElement('custom-table'));
```

Lecturas:
* https://developer.mozilla.org/en-US/docs/Web/API/Web_components: Guia principal de referencia. 
* https://github.com/mdn/web-components-examples: Ejemplos de Custom Elements de la MDN.
* https://es.javascript.info/web-components Otro muy buen manual. 
* https://learn-wcs.com/?active-item=why-web-components Colección de enlaces muy buena. 